In [52]:
import pandas as pd
commands=pd.read_csv('TextCommands.csv')

In [53]:
commands.columns = ['text','label','misc']
commands.head()

,text,label,misc
0,Undo the last sentence,1,NaN
1,Undo the last word,1,NaN
2,Can you undo the last sentence,1,NaN
3,Please undo the text,1,NaN
4,Undo the selected text,1,NaN


In [54]:
commands['text'].head()

0            Undo the last sentence
1                Undo the last word
2    Can you undo the last sentence
3              Please undo the text
4            Undo the selected text
Name: text, dtype: object

In [55]:
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

MAX_SEQUENCE_LENGTH = 10
MAX_NUM_WORDS = 5000

tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(commands['text'])
sequences = tokenizer.texts_to_sequences(commands['text'])

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(commands['label']))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Found 142 unique tokens.
Shape of data tensor: (399, 10)
Shape of label tensor: (399, 27)


In [56]:
import json

with open( 'android/word_dict.json' , 'w' ) as file:
	json.dump( tokenizer.word_index , file )

In [57]:
VALIDATION_SPLIT = 0.1
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]

In [58]:
x_train.shape

(360, 10)

In [59]:
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding, Flatten
from keras.models import Model
from keras.models import Sequential
from keras.initializers import Constant
EMBEDDING_DIM = 60
num_words = min(MAX_NUM_WORDS, len(word_index) + 1)
embedding_layer = Embedding(num_words,EMBEDDING_DIM,input_length=MAX_SEQUENCE_LENGTH,trainable=True)

In [60]:
print('Training model.')
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(64, 3, activation='relu')(embedded_sequences)
x = Conv1D(64, 3, activation='relu')(x)
x = MaxPooling1D(2)(x)

x=Flatten()(x)
x = Dense(100, activation='relu')(x)
preds = Dense(27, activation='softmax')(x)

model = Model(sequence_input, preds)

Training model.


In [61]:
model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['acc'])

In [62]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 10)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 10, 60)            8580      
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 8, 64)             11584     
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 6, 64)             12352     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 3, 64)             0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 192)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               1930

In [63]:
s=0.0
for i in range (1,50):
    model.fit(x_train, y_train,batch_size=50, epochs=30, validation_data=(x_val, y_val))
    # evaluate the model
    scores = model.evaluate(x_val, y_val, verbose=0)
    s=s+(scores[1]*100)

Train on 360 samples, validate on 39 samples
Epoch 1/30
360/360 [==============================] - 0s 684us/step - loss: 3.2864 - acc: 0.0556 - val_loss: 3.2641 - val_acc: 0.1795
Epoch 2/30
360/360 [==============================] - 0s 105us/step - loss: 3.2279 - acc: 0.1833 - val_loss: 3.2222 - val_acc: 0.2051
Epoch 3/30
360/360 [==============================] - 0s 83us/step - loss: 3.1181 - acc: 0.1861 - val_loss: 3.1242 - val_acc: 0.1538
Epoch 4/30
360/360 [==============================] - 0s 89us/step - loss: 2.9459 - acc: 0.2306 - val_loss: 3.0127 - val_acc: 0.1795
Epoch 5/30
360/360 [==============================] - 0s 86us/step - loss: 2.7394 - acc: 0.3000 - val_loss: 2.8670 - val_acc: 0.2308
Epoch 6/30
360/360 [==============================] - 0s 111us/step - loss: 2.5200 - acc: 0.3444 - val_loss: 2.7432 - val_acc: 0.2051
Epoch 7/30
360/360 [==============================] - 0s 89us/step - loss: 2.2806 - acc: 0.4000 - val_loss: 2.5895 - val_acc: 0.2308
Epoch 8/30
360/360 [=

360/360 [==============================] - 0s 86us/step - loss: 2.9239e-04 - acc: 1.0000 - val_loss: 0.4816 - val_acc: 0.9487
Epoch 2/30
360/360 [==============================] - 0s 86us/step - loss: 2.7145e-04 - acc: 1.0000 - val_loss: 0.4776 - val_acc: 0.9487
Epoch 3/30
360/360 [==============================] - 0s 83us/step - loss: 1.6888e-04 - acc: 1.0000 - val_loss: 0.4621 - val_acc: 0.9231
Epoch 4/30
360/360 [==============================] - 0s 86us/step - loss: 1.5296e-04 - acc: 1.0000 - val_loss: 0.5013 - val_acc: 0.9231
Epoch 5/30
360/360 [==============================] - 0s 91us/step - loss: 0.0245 - acc: 0.9944 - val_loss: 0.5513 - val_acc: 0.8718
Epoch 6/30
360/360 [==============================] - 0s 86us/step - loss: 0.0035 - acc: 0.9972 - val_loss: 0.4198 - val_acc: 0.9487
Epoch 7/30
360/360 [==============================] - 0s 81us/step - loss: 1.5776e-04 - acc: 1.0000 - val_loss: 0.4219 - val_acc: 0.9231
Epoch 8/30
360/360 [==============================] - 0s 81u

360/360 [==============================] - 0s 83us/step - loss: 1.7816e-06 - acc: 1.0000 - val_loss: 0.6091 - val_acc: 0.9487
Train on 360 samples, validate on 39 samples
Epoch 1/30
360/360 [==============================] - 0s 94us/step - loss: 1.2387e-06 - acc: 1.0000 - val_loss: 0.6109 - val_acc: 0.9487
Epoch 2/30
360/360 [==============================] - 0s 86us/step - loss: 8.4071e-07 - acc: 1.0000 - val_loss: 0.5984 - val_acc: 0.9487
Epoch 3/30
360/360 [==============================] - 0s 81us/step - loss: 5.9934e-07 - acc: 1.0000 - val_loss: 0.6028 - val_acc: 0.9487
Epoch 4/30
360/360 [==============================] - 0s 89us/step - loss: 4.4934e-07 - acc: 1.0000 - val_loss: 0.6120 - val_acc: 0.9487
Epoch 5/30
360/360 [==============================] - 0s 86us/step - loss: 3.3113e-07 - acc: 1.0000 - val_loss: 0.6137 - val_acc: 0.9487
Epoch 6/30
360/360 [==============================] - 0s 100us/step - loss: 2.4769e-07 - acc: 1.0000 - val_loss: 0.6043 - val_acc: 0.9487
Epoch 

Epoch 29/30
360/360 [==============================] - 0s 84us/step - loss: 2.1884e-06 - acc: 1.0000 - val_loss: 0.6356 - val_acc: 0.9487
Epoch 30/30
360/360 [==============================] - 0s 83us/step - loss: 1.5536e-06 - acc: 1.0000 - val_loss: 0.6340 - val_acc: 0.9487
Train on 360 samples, validate on 39 samples
Epoch 1/30
360/360 [==============================] - 0s 86us/step - loss: 1.1659e-06 - acc: 1.0000 - val_loss: 0.6327 - val_acc: 0.9487
Epoch 2/30
360/360 [==============================] - 0s 83us/step - loss: 8.9240e-07 - acc: 1.0000 - val_loss: 0.6316 - val_acc: 0.9487
Epoch 3/30
360/360 [==============================] - 0s 81us/step - loss: 6.9439e-07 - acc: 1.0000 - val_loss: 0.6306 - val_acc: 0.9487
Epoch 4/30
360/360 [==============================] - 0s 83us/step - loss: 5.5929e-07 - acc: 1.0000 - val_loss: 0.6300 - val_acc: 0.9487
Epoch 5/30
360/360 [==============================] - 0s 83us/step - loss: 4.5432e-07 - acc: 1.0000 - val_loss: 0.6293 - val_acc: 0

Epoch 28/30
360/360 [==============================] - 0s 89us/step - loss: 8.2453e-08 - acc: 1.0000 - val_loss: 0.6759 - val_acc: 0.9744
Epoch 29/30
360/360 [==============================] - 0s 83us/step - loss: 6.8214e-08 - acc: 1.0000 - val_loss: 0.6398 - val_acc: 0.9744
Epoch 30/30
360/360 [==============================] - 0s 86us/step - loss: 5.0995e-08 - acc: 1.0000 - val_loss: 0.5831 - val_acc: 0.9744
Train on 360 samples, validate on 39 samples
Epoch 1/30
360/360 [==============================] - 0s 97us/step - loss: 4.2386e-08 - acc: 1.0000 - val_loss: 0.6267 - val_acc: 0.9744
Epoch 2/30
360/360 [==============================] - 0s 92us/step - loss: 2.9802e-08 - acc: 1.0000 - val_loss: 0.6260 - val_acc: 0.9744
Epoch 3/30
360/360 [==============================] - 0s 86us/step - loss: 2.3842e-08 - acc: 1.0000 - val_loss: 0.6288 - val_acc: 0.9744
Epoch 4/30
360/360 [==============================] - 0s 103us/step - loss: 1.8213e-08 - acc: 1.0000 - val_loss: 0.6211 - val_acc:

...

Epoch 11/30
360/360 [==============================] - 0s 86us/step - loss: 7.7155e-08 - acc: 1.0000 - val_loss: 0.7633 - val_acc: 0.9744
Epoch 12/30
360/360 [==============================] - 0s 89us/step - loss: 5.6624e-08 - acc: 1.0000 - val_loss: 0.7636 - val_acc: 0.9744
Epoch 13/30
360/360 [==============================] - 0s 83us/step - loss: 4.0399e-08 - acc: 1.0000 - val_loss: 0.7645 - val_acc: 0.9744
Epoch 14/30
360/360 [==============================] - 0s 106us/step - loss: 2.7815e-08 - acc: 1.0000 - val_loss: 0.7648 - val_acc: 0.9744
Epoch 15/30
360/360 [==============================] - 0s 83us/step - loss: 2.1193e-08 - acc: 1.0000 - val_loss: 0.7662 - val_acc: 0.9744
Epoch 16/30
360/360 [==============================] - 0s 103us/step - loss: 1.4901e-08 - acc: 1.0000 - val_loss: 0.7671 - val_acc: 0.9744
Epoch 17/30
360/360 [==============================] - 0s 89us/step - loss: 1.0596e-08 - acc: 1.0000 - val_loss: 0.7681 - val_acc: 0.9744
Epoch 18/30
360/360 [===========

Epoch 10/30
360/360 [==============================] - 0s 83us/step - loss: 1.6557e-09 - acc: 1.0000 - val_loss: 0.8102 - val_acc: 0.9744
Epoch 11/30
360/360 [==============================] - 0s 83us/step - loss: 6.6227e-10 - acc: 1.0000 - val_loss: 0.8111 - val_acc: 0.9744
Epoch 12/30
360/360 [==============================] - 0s 83us/step - loss: 3.3114e-10 - acc: 1.0000 - val_loss: 0.8101 - val_acc: 0.9744
Epoch 13/30
360/360 [==============================] - 0s 84us/step - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 0.8111 - val_acc: 0.9744
Epoch 14/30
360/360 [==============================] - 0s 81us/step - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 0.8122 - val_acc: 0.9744
Epoch 15/30
360/360 [==============================] - 0s 89us/step - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 0.8191 - val_acc: 0.9744
Epoch 16/30
360/360 [==============================] - 0s 92us/step - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 0.8203 - val_acc: 0.9744
Epoch 17/30
360/360 [=============

Epoch 9/30
360/360 [==============================] - 0s 86us/step - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 0.9882 - val_acc: 0.9744
Epoch 10/30
360/360 [==============================] - 0s 81us/step - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 0.9883 - val_acc: 0.9744
Epoch 11/30
360/360 [==============================] - 0s 81us/step - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 0.9882 - val_acc: 0.9744
Epoch 12/30
360/360 [==============================] - 0s 81us/step - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 0.9881 - val_acc: 0.9744
Epoch 13/30
360/360 [==============================] - 0s 89us/step - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 0.9881 - val_acc: 0.9744
Epoch 14/30
360/360 [==============================] - 0s 94us/step - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 0.9877 - val_acc: 0.9744
Epoch 15/30
360/360 [==============================] - 0s 86us/step - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 0.9880 - val_acc: 0.9744
Epoch 16/30
360/360 [==============

Epoch 8/30
360/360 [==============================] - 0s 97us/step - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 0.9333 - val_acc: 0.9744
Epoch 9/30
360/360 [==============================] - 0s 86us/step - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 0.9063 - val_acc: 0.9744
Epoch 10/30
360/360 [==============================] - 0s 81us/step - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 0.9046 - val_acc: 0.9744
Epoch 11/30
360/360 [==============================] - 0s 83us/step - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 0.9023 - val_acc: 0.9744
Epoch 12/30
360/360 [==============================] - 0s 97us/step - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 0.9002 - val_acc: 0.9744
Epoch 13/30
360/360 [==============================] - 0s 83us/step - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 0.8979 - val_acc: 0.9744
Epoch 14/30
360/360 [==============================] - 0s 83us/step - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 0.8943 - val_acc: 0.9744
Epoch 15/30
360/360 [===============

Epoch 7/30
360/360 [==============================] - 0s 83us/step - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 1.1071 - val_acc: 0.9744
Epoch 8/30
360/360 [==============================] - 0s 89us/step - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 1.0515 - val_acc: 0.9744
Epoch 9/30
360/360 [==============================] - 0s 86us/step - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 1.0490 - val_acc: 0.9744
Epoch 10/30
360/360 [==============================] - 0s 83us/step - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 1.0452 - val_acc: 0.9744
Epoch 11/30
360/360 [==============================] - 0s 83us/step - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 1.0399 - val_acc: 0.9744
Epoch 12/30
360/360 [==============================] - 0s 97us/step - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 1.0343 - val_acc: 0.9744
Epoch 13/30
360/360 [==============================] - 0s 83us/step - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 1.0238 - val_acc: 0.9744
Epoch 14/30
360/360 [================

In [64]:
s/50

94.20512843132019

In [65]:
# evaluate the model
scores = model.evaluate(x_val, y_val, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


acc: 94.87%


In [66]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [67]:
from keras.models import model_from_json
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = loaded_model.evaluate(x_val, y_val, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

Loaded model from disk
accuracy: 94.87%


In [68]:

# new instance where we do not know the answer
Xnew=["kindly undo the changes","Can you please undo the last paragraph","Make bold this","Would you be kind enough to bold the last word?","Please remove bold from the last paragraph","Kindly unbold the selected text","Kindly insert comment here","Can you please put a comment here","Can you please centre align this text","Can you please position this text in the middle"]

sequences_new = tokenizer.texts_to_sequences(Xnew)

data = pad_sequences(sequences_new, maxlen=MAX_SEQUENCE_LENGTH)

# make a prediction
yprob = model.predict(data)
yclasses=yprob.argmax(axis=-1)
# show the inputs and predicted outputs
print("X=%s, Predicted=%s\nX=%s, Predicted=%s\nX=%s, Predicted=%s\nX=%s, Predicted=%s\nX=%s, Predicted=%s\nX=%s, Predicted=%s\nX=%s, Predicted=%s\nX=%s, Predicted=%s\nX=%s, Predicted=%s\nX=%s, Predicted=%s" % (Xnew[0], yclasses[0],Xnew[1],yclasses[1],Xnew[2],yclasses[2],Xnew[3],yclasses[3],Xnew[4],yclasses[4],Xnew[5],yclasses[5],Xnew[6],yclasses[6],Xnew[7],yclasses[7],Xnew[8],yclasses[8],Xnew[9],yclasses[9]))

X=kindly undo the changes, Predicted=1
X=Can you please undo the last paragraph, Predicted=1
X=Make bold this, Predicted=2
X=Would you be kind enough to bold the last word?, Predicted=2
X=Please remove bold from the last paragraph, Predicted=5
X=Kindly unbold the selected text, Predicted=3
X=Kindly insert comment here, Predicted=15
X=Can you please put a comment here, Predicted=15
X=Can you please centre align this text, Predicted=14
X=Can you please position this text in the middle, Predicted=14


In [48]:
# Converting a saved keras model to a Tensorflow Lite model
import tensorflow as tf
import keras
keras_file = "models/model.h5"
keras.models.save_model(model,keras_file)
converter = tf.lite.TFLiteConverter.from_keras_model_file(keras_file)
converter.post_training_quantize = True
tfmodel = converter.convert()
open("android/model.tflite","wb").write(tfmodel)

INFO:tensorflow:Froze 9 variables.
INFO:tensorflow:Converted 9 variables to const ops.


C:\Users\t-kodasg\Anaconda3\envs\CNN\lib\site-packages\tensorflow\lite\python\lite.py:769: UserWarning: Property post_training_quantize is deprecated, please use optimizations=[Optimize.DEFAULT] instead.
  " instead." % name)


58688